In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Étape 1 : Chargement des données

In [2]:
def load_data(file_path, sep=",", encoding="utf-8", header=None):
    """
    Charge les données depuis un fichier CSV.
    """
    try:
        data = pd.read_csv(file_path, sep=sep, encoding=encoding, header=header)
        print(f"Données chargées avec succès depuis '{file_path}'.")
        return data
    except Exception as e:
        print(f"Erreur lors du chargement des données : {e}")
        return None

# Étape 2 : Sélection des colonnes à conserver 


In [ ]:
def keep_only_necessary_columns(data, columns_to_keep):
    """
    Garde uniquement les colonnes spécifiées dans le DataFrame.
    """
    try:
        initial_columns = data.columns.tolist()
        data = data[columns_to_keep]
        print(f"\nColonnes conservées : {columns_to_keep}")
        dropped_columns = [col for col in initial_columns if col not in columns_to_keep]
        if dropped_columns:
            print(f"Colonnes supprimées : {dropped_columns}")
        return data
    except Exception as e:
        print(f"Erreur lors de la sélection des colonnes : {e}")
        return data

# Étape 3 : Nettoyage des données 

In [ ]:
def clean_data(data, rules):
    """
    Effectue le nettoyage des données selon les règles spécifiques.
    """
    # Renommage des colonnes si nécessaire
    if "rename_columns" in rules:
        data.columns = rules["rename_columns"]
        print("Colonnes renommées.")

    # Suppression des doublons
    data = data.drop_duplicates()
    print(f"Doublons supprimés. Nouvelle taille : {data.shape}")

    # Remplacement des valeurs manquantes
    numeric_cols = data.select_dtypes(include=[np.number]).columns
    categorical_cols = data.select_dtypes(include=["object"]).columns

    for col in numeric_cols:
        if col in rules.get("numeric_fill", {}):
            data[col].fillna(rules["numeric_fill"][col], inplace=True)
        else:
            data[col].fillna(0, inplace=True)  # Valeur par défaut

    for col in categorical_cols:
        data[col].fillna("Non spécifié", inplace=True)

    # Identification et suppression des anomalies
    if "anomalies" in rules:
        anomalies = data.query(rules["anomalies"])
        if not anomalies.empty:
            print(f"{len(anomalies)} anomalies détectées.")
            data = data[~data.index.isin(anomalies.index)]

    print("Nettoyage des données terminé.")
    return data

# Étape 4 : Transformation des données 

In [ ]:
def transform_data(data, rules):
    """
    Transforme les données : encodage, création de nouvelles features, etc.
    """
    # Encodage des variables catégoriques
    if "categorical_columns" in rules:
        for col in rules["categorical_columns"]:
            le = LabelEncoder()
            data[col] = le.fit_transform(data[col].astype(str))
        print("Encodage des variables catégoriques terminé.")

    # Conversion des dates en format datetime
    if "date_columns" in rules:
        for col in rules["date_columns"]:
            data[col] = pd.to_datetime(data[col], errors="coerce")

    # Création de nouvelles features
    if "new_features" in rules:
        for feature_name, feature_formula in rules["new_features"].items():
            data[feature_name] = eval(feature_formula)
        print("Création de nouvelles features terminée.")

    print("Transformation des données terminée.")
    return data

# Étape 5 : Validation des données 

In [ ]:
def validate_data(data):
    """
    Valide les données pour s'assurer qu'elles sont prêtes pour l'analyse.
    """
    # Vérification des valeurs manquantes
    missing_values = data.isnull().sum().sum()
    if missing_values > 0:
        print(f"Attention : {missing_values} valeurs manquantes restantes.")
    else:
        print("Aucune valeur manquante dans les données.")

    # Vérification des types de données
    print("Types de données dans le DataFrame :")
    print(data.dtypes)

    print("Validation des données terminée.")

# Pipeline principal 

In [ ]:
def preprocess_data(file_path, output_file, rules):
    """
    Prétraite les données selon les règles spécifiques.
    """
    # Étape 1 : Chargement des données
    data = load_data(file_path, **rules.get("load_params", {}))
    if data is None:
        return None

    # Étape 2 : Sélection des colonnes à conserver
    if "columns_to_keep" in rules:
        data = keep_only_necessary_columns(data, rules["columns_to_keep"])

    # Étape 3 : Nettoyage des données
    data = clean_data(data, rules)

    # Étape 4 : Transformation des données
    data = transform_data(data, rules)

    # Étape 5 : Validation des données
    validate_data(data)

    # Sauvegarde des données prétraitées
    data.to_csv(output_file, index=False)
    print(f"Données prétraitées sauvegardées dans '{output_file}'.")

    return data

# Configuration des règles pour chaque fichier 

In [ ]:
RULES = {
    "Article": {
        "load_params": {"header": 0},
        "columns_to_keep": ["ID_Article", "Nom_Article", "Prix_vente_TTC", "Prix_achat_HT",
                            "Categorie", "Quantite_Stock", "Date_Creation"],
        "numeric_fill": {"Prix_vente_TTC": 0, "Prix_achat_HT": 0},
        "anomalies": "(Prix_vente_TTC < 0) | (Prix_achat_HT < 0)",
        "categorical_columns": ["Categorie"],
        "date_columns": ["Date_Creation"],
        "new_features": {
            "Marge_Brute": "data['Prix_vente_TTC'] - data['Prix_achat_HT']",
            "Taux_Marge": "data['Marge_Brute'] / data['Prix_achat_HT'].replace(0, np.nan)"
        }
    },
    "Dispos": {
        "load_params": {"header": 0},
        "columns_to_keep": ["GQ_STOCKMIN", "GQ_STOCKMAX", "GQ_PHYSIQUE", "GQ_DATECREATION"],
        "numeric_fill": {"GQ_STOCKMIN": 0, "GQ_STOCKMAX": "data['GQ_PHYSIQUE'] * 2"},
        "anomalies": "(GQ_PHYSIQUE > GQ_STOCKMAX) | (GQ_PHYSIQUE < GQ_STOCKMIN)",
        "date_columns": ["GQ_DATECREATION"],
        "new_features": {
            "Taux_Remplissage": "data['GQ_PHYSIQUE'] / data['GQ_STOCKMAX'].replace(0, np.nan)",
            "Stock_Disponible": "data['GQ_PHYSIQUE'] - data['GQ_STOCKMIN']"
        }
    },
    "Tiers": {
        "load_params": {"header": None},
        "columns_to_keep": ["T_AUXILIAIRE", "T_TIERS", "T_DEVISE", "T_NATURE", "T_REMISE",
                            "T_VILLE", "T_PAYS", "T_DATECREATION", "T_DATEFERMETURE"],
        "numeric_fill": {},
        "anomalies": "(T_DATECREATION > T_DATEFERMETURE) & (T_DATEFERMETURE != '1900-01-01 00:00:00')",
        "categorical_columns": ["T_DEVISE", "T_NATURE", "T_VILLE", "T_PAYS"],
        "date_columns": ["T_DATECREATION", "T_DATEFERMETURE"],
        "new_features": {
            "Anciennete": "(pd.Timestamp.now() - data['T_DATECREATION']).dt.days / 365",
            "Statut_Actif": "np.where(data['T_DATEFERMETURE'] == '1900-01-01 00:00:00', 1, 0)"
        }
    }
}

#  Exécution principale 

In [ ]:
if __name__ == "__main__":
    # Liste des fichiers à prétraiter
    files_to_process = [
        {"file_path": "Article.csv", "output_file": "Article_Preprocessed.csv", "rules": RULES["Article"]},
        {"file_path": "DISPOS.csv", "output_file": "DISPO_Preprocessed.csv", "rules": RULES["Dispos"]},
        {"file_path": "Tier.csv", "output_file": "TIERS_Preprocessed.csv", "rules": RULES["Tiers"]}
    ]

    # Prétraitement de chaque fichier
    for file_info in files_to_process:
        print(f"\n=== Traitement du fichier : {file_info['file_path']} ===")
        preprocess_data(
            file_path=file_info["file_path"],
            output_file=file_info["output_file"],
            rules=file_info["rules"]
        )